In [3]:
import json
import pandas as pd
import pygame, sys, random, time

with open('dataset.json') as f:
#opens the dataset obtained at https://www.kaggle.com/theriley106/hq-trivia-question-database
    data = json.load(f) 

#parse data into a friendlier output, and filters alternatives that are not complete
df = pd.json_normalize(data)
df = df[df['answers'].str.len() == 3] # filters out questions that do not have 3 answer alternatives
df = df.reset_index(drop=True)

In [4]:
def get_questions_alternatives(game_state):
    ''' 
    Defines a random question from the dataframe, by calculating a random ID
    '''
    if game_state == 'main_game':
        rand_i = random.choice(id_list)
        id_list.remove(rand_i)
        

        return rand_i

def display_questions(rand_i):
    ''' 
    Displays a random question from the dataframe, and its alternatives
    '''
    question_form = game_font.render(df['question'][rand_i],True,(255,255,255))
    alternative1_form = game_font.render(df['answers'][rand_i][0]['text'],True,(255,255,255))
    alternative2_form = game_font.render(df['answers'][rand_i][1]['text'],True,(255,255,255))
    alternative3_form = game_font.render(df['answers'][rand_i][2]['text'],True,(255,255,255))

    question_rect = question_form.get_rect(center = (385,221))
    alternative1_rect = alternative1_form.get_rect(center = (240,307))
    alternative2_rect = alternative2_form.get_rect(center = (565, 307))
    alternative3_rect = alternative3_form.get_rect(center = (390,375))

    screen.blit(question_form,question_rect)    
    screen.blit(alternative1_form,alternative1_rect) 
    screen.blit(alternative2_form,alternative2_rect) 
    screen.blit(alternative3_form,alternative3_rect)
        

def check_correct(select_choice, rand_i):
    ''' 
    Checks if player selected the right alternative
    '''
    global game_active, score
    pygame.mixer.music.load('media/music/suspense.mp3')
    pygame.mixer.music.play(0)
    time.sleep(4)

    if df['answers'][rand_i][select_choice]['correct'] == True:
        path_song = 'media/music/certa-resposta.mp3'
        score += 10
    else:
        path_song = 'media/music/errou.mp3'       
        score = 0
    
    game_active = True        
    pygame.mixer.music.load(path_song)
    pygame.mixer.music.play(0)
    time.sleep(1.5)

def display_score(score):
    ''' 
    Changes and displays score obtained by the player on the current game
    '''
    score_form = game_font.render(f'Score: {score}',True,(0,0,0))
    score_rect = score_form.get_rect(center = (400,165))
    screen.blit(score_form,score_rect)

def display_max_score(max_score):
    ''' 
    Changes and displays maximum score obtained by the player on the current running instance
    '''

    global score
    if max_score < score:
        max_score = score
    
    max_score_form = game_font.render(f'Max Score: {max_score}',True,(0,0,0))
    max_score_rect = max_score_form.get_rect(center = (75,425))
    screen.blit(max_score_form,max_score_rect)    
    return max_score

def change_background(img):
    ''' 
    Changes the background according to the alternative chosen
    '''
    global rand_i
    screen.blit(img,(0,0)) 
    display_questions(rand_i)
    pygame.display.update()

## Pygame initiation and its relevant variables' definition  
screen_WIDTH = 451
screen_LENGTH = 800
font_SIZE = 14

pygame.init()
pygame.display.set_caption('Show do Milhão - versão 1')
screen = pygame.display.set_mode((screen_LENGTH, screen_WIDTH))
clock = pygame.time.Clock()
game_font = pygame.font.Font('media/font/arial.ttf',font_SIZE)

pygame.mixer.music.load('media/music/abertura.mp3')
pygame.mixer.music.play(0)

bg_surface = pygame.image.load('media/img/0-main_window.jpg').convert() #main background pic
bg_surface_a = pygame.image.load('media/img/00-main_window.jpg').convert()
bg_surface_b = pygame.image.load('media/img/01-main_window.jpg').convert()
bg_surface_c = pygame.image.load('media/img/02-main_window.jpg').convert()

# game first variables
game_active = True
score = 0 #get score of current game
max_score = 0 #get maximum score obtained on the given instance
current_i = 0 #get the current question position at the df
select_choice = 5 #get the player's alternative choice
bye_bye = False

#generate a list of question IDs to get random questions
id_list = [i for i in range(0,df.shape[0])] 

while not bye_bye:                
    screen.blit(bg_surface,(0,0))

    if game_active:
        rand_i = get_questions_alternatives('main_game')
        game_active = False #set to False to keep question at the screen
    
    display_questions(rand_i)
    display_score(score)
    max_score = display_max_score(max_score)
    pygame.display.update()
    
    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_a: #player selects first alternative
                change_background(bg_surface_a) 
                select_choice = 0
            if event.key == pygame.K_b: #player selects second alternative
                change_background(bg_surface_b)
                select_choice = 1                   
            if event.key == pygame.K_c: #player selects third alternative
                change_background(bg_surface_c)
                select_choice = 2
            if event.key == pygame.K_ESCAPE: #bye bye
                pygame.quit()
#                 sys.exit(1)
                bye_bye = True
            pygame.display.update()
            check_correct(select_choice, rand_i)     

        


error: video system not initialized